In [6]:
import tensorflow as tf

model = tf.keras.models.load_model(
    "Face_Mask_Model.h5",
    compile=False
)

print(type(model))


<class 'keras.src.models.functional.Functional'>


In [7]:
model.input_shape


(None, 224, 224, 3)

In [8]:
import cv2
import numpy as np

def preprocess_image(image):
    image = cv2.resize(image, (224, 224))
    image = image.astype("float32") / 255.0
    image = np.expand_dims(image, axis=0)
    return image


In [12]:
img = cv2.imread("test.jpg")
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

processed = preprocess_image(img)
pred = model.predict(processed)[0][0]

label = "Mask" if pred > 0.5 else "No Mask"
confidence = float(pred)

print(label, confidence)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
Mask 0.956886887550354


In [13]:
import os

print(os.path.isfile("maskdetector.model"))
print(os.path.isdir("maskdetector.model"))


True
False


In [18]:
!pip uninstall tensorflow keras -y
!pip install tensorflow==2.12.0


Found existing installation: tensorflow 2.19.1
Uninstalling tensorflow-2.19.1:
  Successfully uninstalled tensorflow-2.19.1
Found existing installation: keras 3.11.2
Uninstalling keras-3.11.2:
  Successfully uninstalled keras-3.11.2


You can safely remove it manually.


  You can safely remove it manually.
  You can safely remove it manually.
  You can safely remove it manually.
  You can safely remove it manually.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
faiss-cpu 1.11.0.post1 requires numpy<3.0,>=1.25.0, but you have numpy 1.23.5 which is incompatible.
onnxslim 0.1.61 requires sympy>=1.13.3, but you have sympy 1.13.1 which is incompatible.
scikit-image 0.25.2 requires numpy>=1.24, but you have numpy 1.23.5 which is incompatible.



INFO: pip is looking at multiple versions of jax to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of jax to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
   ---------------------------------------- 0.0/272.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/272.8 MB 2.0 MB/s eta 0:02:14
   ---------------------------------------- 0.0/272.8 MB 2.0 MB/s eta 0:02:14
   ---------------------------------------- 0.1/272.8 MB 1.1 MB/s eta 0:04:20
   ---------------------------------------- 0.2/272.8 MB 1.3 MB/s eta 0:03:28
   ---------------------------------------- 0.2/272.8 MB 1.3 MB/s eta 0:03:37
   -------

In [6]:
model = tf.keras.models.load_model(
    "mask_detector.h5",
    compile=False
)

print(type(model))


<class 'keras.engine.functional.Functional'>


In [7]:
model.input_shape


(None, 224, 224, 3)

In [8]:
import cv2
import numpy as np

def preprocess_image(image):
    image = cv2.resize(image, (224, 224))
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = image.astype("float32") / 255.0
    image = np.expand_dims(image, axis=0)
    return image


In [13]:
img = cv2.imread("test1.jpg")
processed = preprocess_image(img)

pred = model.predict(processed)[0][0]
print("Raw prediction:", pred)


1/1 [==============================] - 0s 36ms/step
Raw prediction: 0.9972778


In [14]:
import cv2
import numpy as np
import tensorflow as tf

# Load model (already verified)
model = tf.keras.models.load_model("mask_detector.h5", compile=False)

# Face detector
face_cascade = cv2.CascadeClassifier(
    cv2.data.haarcascades + "haarcascade_frontalface_default.xml"
)

# Preprocessing (STRICTLY match model input)
def preprocess_image(image):
    image = cv2.resize(image, (224, 224))
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = image.astype("float32") / 255.0
    image = np.expand_dims(image, axis=0)
    return image

# Webcam
cap = cv2.VideoCapture(0)

THRESHOLD = 0.5  # you can tune to 0.6 if needed

while True:
    ret, frame = cap.read()
    if not ret:
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.3, minNeighbors=5)

    for (x, y, w, h) in faces:
        face = frame[y:y+h, x:x+w]

        try:
            processed = preprocess_image(face)
            pred = model.predict(processed, verbose=0)[0][0]

            if pred > THRESHOLD:
                label = "Mask"
                color = (0, 255, 0)  # Green
            else:
                label = "No Mask"
                color = (0, 0, 255)  # Red

            confidence = pred if label == "Mask" else 1 - pred
            confidence_text = f"{confidence*100:.1f}%"

        except:
            continue

        # Draw bounding box
        cv2.rectangle(frame, (x, y), (x+w, y+h), color, 2)

        # Draw label + confidence
        cv2.putText(
            frame,
            f"{label} ({confidence_text})",
            (x, y-10),
            cv2.FONT_HERSHEY_SIMPLEX,
            0.8,
            color,
            2
        )

    cv2.imshow("Face Mask Detection", frame)

    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()
